In order to successfully complete this assignment you must do the required reading, watch the provided videos and complete all instructions.  The embedded Google form must be entirely filled out and submitted on or before **11:59pm on Thursday March 14th**.  Students must come to class the next day prepared to discuss the material covered in this assignment.

# Pre-Class Assignment: Parallel Python

### Goals for today's pre-class assignment 

</p>

1. The Python GIL (Global Interface Lock)
2. Getting around the GIL

# 1. Parallel Python example

Here is an example for running parallel python using the ```multiprocessing``` library. 

https://stackoverflow.com/questions/10415028/how-can-i-recover-the-return-value-of-a-function-passed-to-multiprocessing-proce


In [ ]:
import multiprocessing
num_procs = multiprocessing.cpu_count()
print('You have', num_procs, 'processors')

def worker(procnum, return_dict):
    '''worker function'''
    print(str(procnum) + ' represent!')
    return_dict[procnum] = procnum


if __name__ == '__main__':
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,return_dict))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
    print(return_dict.values())

### Lets try to make a parallel matrix multiply

The following is the instructor's attempt at using multiprocessing to do matrix multiply. First lets start with a serial method.

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import sympy as sp
import random
import time
sp.init_printing(use_unicode=True)

In [ ]:
#simple matrix multiply (no numpy)
def multiply(m1,m2):
    m = len(m1)
    d = len(m2)
    n = len(m2[0])
    if len(m1[0]) != d:
        print("ERROR - inner dimentions not equal")
    result = [[0 for i in range(m)] for j in range(n)]
    for i in range(0,m):
        for j in range(0,n):
            for k in range(0,d):
                result[i][j] = result[i][j] + m1[i][k] * m2[k][j]
    return result

In [ ]:
# Random generated 2d lists of lists that can be multiplied 
m = 4
d = 10
n = 4

A = [[random.random() for i in range(d)] for j in range(m)]
B = [[random.random() for i in range(n)] for j in range(d)]

In [ ]:
#Compute matrix multiply using your function


start = time.time()

simple_answer = multiply(A, B)
simple_time = time.time()-start

print('simple_answer =',simple_time,'seconds')

Lets compare this to the numpy result:

In [ ]:
#Compare to numpy result
start = time.time()

np_answer = np.matrix(A)*np.matrix(B)
np_time = time.time()-start

print('np_answer =',np_time,'seconds')


In [ ]:
#Compare to numpy result
A_ = np.matrix(A)
B_ = np.matrix(B)

start = time.time()

np_answer = A_*B_
np_time = time.time()-start

print('np_answer =',np_time,'seconds')

In [ ]:
np.allclose(simple_answer,np_answer)

On some systems the numpy result may be slower than the simple result.  Think about why this might be.  We will discuss this later. 

### Now lets use multiprocessing to try and do a parallel method

In [ ]:
#Attempt at a parallel multiply
def parallel_multiply(m1,m2):
    m = len(m1)
    d = len(m2)
    n = len(m2[0])

    def dot_worker(row,col):
        """thread worker function"""
        #print('Worker:', i,j)
        temp = 0
        for k in range(len(m2)):
            temp = temp + m1[row][k] * m2[k][col]
        return_dict[(row,col)] = temp
        return 

    jobs = []
    manager = multiprocessing.Manager()
    return_dict = manager.dict()

    for i in range(m):
        for j in range(n):
            #p = dot_worker(i,j)
            p = multiprocessing.Process(target=dot_worker, args=(i,j,))
            jobs.append(p)
            p.start()

    for proc in jobs:
        proc.join()
    
    print('Used',len(jobs),'threads in calculation.')
    
    C = return_dict.values()
    C = np.matrix(return_dict.values())
    C = C.reshape((m,n))
    return C

In [ ]:
#Compute matrix multiply using your function
start = time.time()

parallel_answer = parallel_multiply(A, B)
parallel_time = time.time()-start

print('parallel_answer=',parallel_time,'seconds')

In [ ]:
np.allclose(parallel_answer,np_answer)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
objects = ('Simple', 'Numpy', 'parallel')
y_pos = np.arange(len(objects))
performance = [simple_time,np_time,parallel_time]
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Time (seconds)')
plt.yscale('log')
plt.title('Programming language usage')
 

&#9989; <font color=red>**QUESTION:**</font> Why do you think the parallel version was so much slower than Python?

Put your answer to the above question here.

# 2. The Python GIL (Global Interface Lock)


&#9989; <font color=red>**DO THIS:**</font> Read the following blog post and answer the questions: https://wiki.python.org/moin/GlobalInterpreterLock

&#9989; <font color=red>**QUESTION:**</font>  Why was the GIL introduced to the Python programming language?

Put your answer to the above question here.

&#9989; <font color=red>**QUESTION:**</font>  How does the GIL help avoid race conditions?

Put your answer to the above question here.

&#9989; <font color=red>**QUESTION:**</font>  How does the GIL help avoid deadlock?

Put your answer to the above question here.

&#9989; <font color=red>**QUESTION:**</font>  Why is the GIL problematic to parallel libraries like the "thread" and "multiprocessing" libraries?

Put your answer to the above question here.

# 3. Getting around the GIL

Fortunately there are ways to get around the GIL. In fact, Python has libraries that do shared memory parallelization, shared network parallelization and GPU acceleration.  Do some research and answer the following questions:


&#9989; <font color=red>**QUESTION:**</font>  Some of ```numpy``` library can run in parallel. How does ```numpy``` get around the GIL? 

Put your answer to the above question here.

&#9989; <font color=red>**QUESTION:**</font>  The  ```numba``` library can also run in parallel. How does ```numba``` get around the GIL? 

Put your answer to the above question here.

&#9989; <font color=red>**QUESTION:**</font> What python library can be used to program GPUs?

Put your answer to the above question here.

&#9989; <font color=red>**QUESTION:**</font> What python library can be used to run shared network parallelization such as the Message Passing Interface (MPI)?

Put your answer to the above question here.

&#9989; <font color=red>**QUESTION:**</font> There seem to be a lot of solutions for running Python in parallel.  Provide an argument(s) as to why you would bother with an "older" language such as C/C++ or Fortran? 

Put your answer to the above question here.

----
# 5. Assignment wrap-up

Please fill out the form that appears when you run the code below.  **You must completely fill this out in order to receive credit for the assignment!**

[Direct Link](https://docs.google.com/forms/d/e/1FAIpQLScZALQfY4CD6nCcIwZyAWFaE_E0Yf520BPStYubspxVO72tgg/viewform)

&#9989; <font color=red>**QUESTION:**</font> Approximately how long (in minutes) did this assignment take for you to complete.

Put your answer to the above question here

&#9989; <font color=red>**QUESTION:**</font>  What questions do you have, if any, about any of the topics discussed in this assignment after working through the jupyter notebook?

Put your answer to the above question here

&#9989; <font color=red>**QUESTION:**</font>  Do you have any further questions or comments about this material, or anything else that's going on in class? 

Put your answer to the above question here

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://docs.google.com/forms/d/e/1FAIpQLScZALQfY4CD6nCcIwZyAWFaE_E0Yf520BPStYubspxVO72tgg/viewform?embedded=true" 
	width="100%" 
	height="1200px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

---------
### Congratulations, we're done!

To get credit for this assignment you must fill out and submit the above Google From on or before the assignment due date.


**Course Resources:**
- [Syllabus](https://tinyurl.com/y75cnzam)
- [Preliminary Schedule](https://tinyurl.com/CMSE314-Schedule)
- [Git Repository](https://gitlab.msu.edu/colbrydi/cmse401-s19)
- [Jargon Jar and Command History](https://tinyurl.com/CMSE314-JargonJar) 



&#169; Copyright 2019,  Michigan State University Board of Trustees